In [3]:
# To make pyspark importable as a regular library.
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [74]:
import findspark
findspark.init()

In [75]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Assignment").getOrCreate()
type(spark)

pyspark.sql.session.SparkSession

In [27]:
#1.Creating Resilient Distributed Dataset in 3 different ways.

#Method 1:Parallelize

text = "If you look at what you have in life, you'll always have more. If you look at what you don't have in life, you'll never have enough.".split(" ")
words = spark.sparkContext.parallelize(text,2)
typ = type(words)
word = words.collect()
count = words.count()

print("type of words :",typ)
print("Number of words :",count)
print(word)

type of words : <class 'pyspark.rdd.RDD'>
Number of words : 27
['If', 'you', 'look', 'at', 'what', 'you', 'have', 'in', 'life,', "you'll", 'always', 'have', 'more.', 'If', 'you', 'look', 'at', 'what', 'you', "don't", 'have', 'in', 'life,', "you'll", 'never', 'have', 'enough.']


In [58]:
#Method 2: Using Data source

data = spark.sparkContext.textFile("textfile.txt")
Data = data.collect()
t2 = type(Data)
print("type of Data",t2)
print(Data)

type of Data <class 'list'>
["Nature's first green is gold,", 'Her hardest hue to hold.', "Her early leaf's a flower;", 'But only so an hour.', 'Then leaf subsides to leaf.', 'So Eden sank to grief,', 'So dawn goes down to day.', 'Nothing gold can stay.']


In [69]:
#Method 3: Using Pipelined RDD
info = words.filter(lambda word:word.startswith('l'))
inf = info.collect()
type(info)
print(inf)

['look', 'life,', 'look', 'life,']


In [33]:
#2.Read a text file and count the number of words in the file using RDD operations.

Total_words=data.flatMap(lambda s:s.split(" "))
Total_words.count()


40

In [36]:
#3.Write a program to find the word frequency in a given file.
datainfo=spark.sparkContext.textFile("textfile.txt")
datainfo=datainfo.flatMap(lambda x:x.split())
content=datainfo.map(lambda c:(c,1))
content.reduceByKey(lambda a,b:a+b).collect()

[('green', 1),
 ('is', 1),
 ('gold,', 1),
 ('Her', 2),
 ('hardest', 1),
 ('hue', 1),
 ("leaf's", 1),
 ('flower;', 1),
 ('But', 1),
 ('only', 1),
 ('an', 1),
 ('hour.', 1),
 ('Then', 1),
 ('leaf', 1),
 ('subsides', 1),
 ('leaf.', 1),
 ('Eden', 1),
 ('grief,', 1),
 ('dawn', 1),
 ('goes', 1),
 ('down', 1),
 ('Nothing', 1),
 ('gold', 1),
 ("Nature's", 1),
 ('first', 1),
 ('to', 4),
 ('hold.', 1),
 ('early', 1),
 ('a', 1),
 ('so', 1),
 ('So', 2),
 ('sank', 1),
 ('day.', 1),
 ('can', 1),
 ('stay.', 1)]

In [38]:
#4. Write a program to convert all words in a file to uppercase.

data.map(lambda c:c.upper()).collect()

["NATURE'S FIRST GREEN IS GOLD,",
 'HER HARDEST HUE TO HOLD.',
 "HER EARLY LEAF'S A FLOWER;",
 'BUT ONLY SO AN HOUR.',
 'THEN LEAF SUBSIDES TO LEAF.',
 'SO EDEN SANK TO GRIEF,',
 'SO DAWN GOES DOWN TO DAY.',
 'NOTHING GOLD CAN STAY.']

In [39]:
#5. Write a program to convert all words in a file to lowercase.

data.map(lambda c:c.lower()).collect()

["nature's first green is gold,",
 'her hardest hue to hold.',
 "her early leaf's a flower;",
 'but only so an hour.',
 'then leaf subsides to leaf.',
 'so eden sank to grief,',
 'so dawn goes down to day.',
 'nothing gold can stay.']

In [44]:
#6. Write a program to capitalize first letter of each words in file (use string capitalize() method)

Uppercase = data.flatMap(lambda a:a.split(" ")).map(lambda c:c.capitalize()).collect()
" ".join(Uppercase)


"Nature's First Green Is Gold, Her Hardest Hue To Hold. Her Early Leaf's A Flower; But Only So An Hour. Then Leaf Subsides To Leaf. So Eden Sank To Grief, So Dawn Goes Down To Day. Nothing Gold Can Stay."

In [45]:
#7.Find the longest length of word from given set of words.

longestlength=data.flatMap(lambda x:x.split(" "))
longestlength.map(lambda nu:(len(nu),nu)).max()[1]

'subsides'

In [51]:
#8. Map the Registration numbers to corresponding branch.
#6000 series BDA, 9000 series HDA, 1000 series ML, 2000 series VLSI, 3000 series ES, 4000 series MSc, 5000 series CC.
#Given registration number, generate a key-value pair of Registration Number and Corresponding Branch.

USN = [6001,2001,2002,5001,6002,9001,3001,3002,4001,4002,5002,5003,1001,1002]
context=spark.sparkContext.parallelize(USN,2)
classify=context.map(lambda reg:('VLSI',reg) if reg>2000 and reg<3000 
        else ('MS',reg) if reg>1000 and reg<2000
        else ('ES',reg) if reg>3000 and reg<4000
        else ('MSc',reg) if reg>4000 and reg<5000
        else ('CC',reg) if reg>5000 and reg<6000
        else ('BDA',reg) if reg>6000 and reg<7000
        else ('HDA',reg))
classified_usn = classify.collect()
print(classified_usn)

[('BDA', 6001), ('VLSI', 2001), ('VLSI', 2002), ('CC', 5001), ('BDA', 6002), ('HDA', 9001), ('ES', 3001), ('ES', 3002), ('MSc', 4001), ('MSc', 4002), ('CC', 5002), ('CC', 5003), ('MS', 1001), ('MS', 1002)]


In [83]:
#10. A text file (citizen.txt) contains data about citizens of country. Fields (information in file) are Name, dob, Phone,
# email and state name. Another file contains mapping of state names to state code like Karnataka is codes as KA, TamilNadu as TN, Kerala KL etc.
# Compress the citizen.txt file by changing full state name to state code.

Citizen_details = spark.sparkContext.textFile("citizen.txt")
State_code = spark.sparkContext.textFile("State_codes.txt")

details_rdd = Citizen_details.map(lambda x:x.split(",")).collect()
code_rdd= State_code.map(lambda y:y.split(",")).collect()

for i in range(len(details_rdd)):
    for j in range(len(code_rdd)):  
        if details_rdd[i][4]==code_rdd[j][0]:
            details_rdd[i][4]=code_rdd[j][1]
details_rdd     




[['Subrahmanyan', '04-12-1997', '201046007', 'sub@gmail.com', 'KA'],
 ['Abhishek', '15-10-1997', '201051007', 'abhi@gmail.com', 'DL'],
 ['Thejas', '25-5-1997', '201046013', 'thejas@gmail.com', 'AP'],
 ['Rakhesh', '22-03-1997', '201051023', 'rakhi@gmail.com', 'MH'],
 ['Shanthkumar', '14-03-1999', '201046049', 'shanthu@gmail.com', 'GJ'],
 ['kiran', '21-10-1997', '2010156001', 'kiran@gmail.com', 'WestBengal'],
 ['Anupam', '05-09-1993', '201056002', 'anupam@gmail.com', 'RJ'],
 ['Chandan', '13-02-1997', '201046003', 'chandan@gmail.com', 'PB']]

In [85]:
state = spark.sparkContext.textFile('State_codes.txt')
stateKey = state.map(lambda word: (word.split(',')[0], word.split(',')[1]))

Statecode_dict = {}
for val in stateKey.collect():
    Statecode_dict[val[0]] = val[1]
    
Statecode_dict

{'Karnataka': 'KA',
 'Delhi': 'DL',
 'Gujarat': 'GJ',
 'Maharashtra': 'MH',
 'Rajasthan': 'RJ',
 'Punjab': 'PB',
 'AndhraPradesh': 'AP',
 'westBengal': 'WB'}

In [88]:
mapData = spark.sparkContext.broadcast(Statecode_dict)

city = spark.sparkContext.textFile('citizen.txt')
print(city.collect())
def compress(state,codes):
    splitData = state.split(',')  
    print(splitData)
    splitData[4] = codes.value.get(splitData[4])
    newData = ' '
    newData = newData.join(splitData)
    
    return newData
    
Citizen = city.map(lambda data: compress(data,mapData))



['Subrahmanyan,04-12-1997,201046007,sub@gmail.com,Karnataka', 'Abhishek,15-10-1997,201051007,abhi@gmail.com,Delhi', 'Thejas,25-5-1997,201046013,thejas@gmail.com,AndhraPradesh', 'Rakhesh,22-03-1997,201051023,rakhi@gmail.com,Maharashtra', 'Shanthkumar,14-03-1999,201046049,shanthu@gmail.com,Gujarat', 'kiran,21-10-1997,2010156001,kiran@gmail.com,WestBengal', 'Anupam,05-09-1993,201056002,anupam@gmail.com,Rajasthan', 'Chandan,13-02-1997,201046003,chandan@gmail.com,Punjab']
